### Student Information
Name: 許宸瑄

Student ID: B10209001

GitHub ID: ChenHsuanHsu

Kaggle name: kyonatsuki

Kaggle private scoreboard snapshot: 

---

### Instructions

1. First: __This part is worth 30% of your grade.__ Do the **take home** exercises in the [DM2024-Lab2-master Repo](https://github.com/didiersalazar/DM2024-Lab2-Master). You may need to copy some cells from the Lab notebook to this notebook. 


2. Second: __This part is worth 30% of your grade.__ Participate in the in-class [Kaggle Competition](https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework) regarding Emotion Recognition on Twitter by this link: https://www.kaggle.com/competitions/dm-2024-isa-5810-lab-2-homework. The scoring will be given according to your place in the Private Leaderboard ranking: 
    - **Bottom 40%**: Get 20% of the 30% available for this section.

    - **Top 41% - 100%**: Get (60-x)/6 + 20 points, where x is your ranking in the leaderboard (ie. If you rank 3rd your score will be (60-3)/6 + 20 = 29.5% out of 30%)   
    Submit your last submission **BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)**. Make sure to take a screenshot of your position at the end of the competition and store it as '''pic0.png''' under the **img** folder of this repository and rerun the cell **Student Information**.
    

3. Third: __This part is worth 30% of your grade.__ A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 


4. Fourth: __This part is worth 10% of your grade.__ It's hard for us to follow if your code is messy :'(, so please **tidy up your notebook**.


Upload your files to your repository then submit the link to it on the corresponding e-learn assignment.

Make sure to commit and save your changes to your repository __BEFORE the deadline (Nov. 26th 11:59 pm, Tuesday)__. 

### Begin Assignment Here

1. First part of my homework is done in DM2024-Lab2-Master.ipynb under DM2024-Lab2-Master file [open Notebook](https://github.com/ChenHsuanHsu/DM2024-Lab2-Master/blob/main/DM2024-Lab2-Master.ipynb)

2. Second part of my homework is Kaggle.ipynb under same files as this text
[open Notebook](https://github.com/ChenHsuanHsu/DM2024-Lab2-Homework/blob/main/Kaggle.ipynb)

3. Third: A report of your work developping the model for the competition (You can use code and comment it). This report should include what your preprocessing steps, the feature engineering steps and an explanation of your model. You can also mention different things you tried and insights you gained. 
撰寫一份您參加比賽時開發模型的報告（可以使用並註解代碼）。此報告應包含您的數據預處理步驟、特徵工程步驟以及對模型的解釋。您還可以提及您嘗試過的不同方法以及獲得的見解。

## 1. data preprocessing







## 2. feature enginering

## 3. model

## different attempt

[first submission](https://github.com/ChenHsuanHsu/DM2024-Lab2-Homework/blob/main/first_try.ipynb)